In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
boston_dataset=load_boston()

In [3]:
data=pd.DataFrame(data=boston_dataset.data , columns=boston_dataset.feature_names)

In [4]:
# as earlier we found they were not required so removing 
features=data.drop(['INDUS','AGE'], axis=1)
log_price=np.log(boston_dataset.target)
target=pd.DataFrame(log_price,columns=['PRICE'])

In [17]:
# now as few thing people will check while calculating the price estimate so we will 
#create an array of mean values as for the rest values that mean value will be considered

#property_stats if we create will be a series as featres.mean() is a series 
#and series has a attribute called value which is an array
# to create it a 2d array
property_stats=features.mean().values.reshape(1,11)



(1, 11)

In [21]:
# calculting the linear regression 
# perdict return nd array

regr=LinearRegression()
regr.fit(features,target)
fitted_values=regr.predict(features)

# MSE and RMSE
MSE = mean_squared_error(target, fitted_values)
RMSE = np.sqrt(MSE)


In [23]:
fitted_values.shape

(506, 1)

In [30]:
features.shape

(506, 11)

In [27]:
regr.predict(property_stats)

array([[3.03451287]])

In [29]:
# features we areeh user for now to perdict ,
#rest fetures will take up the mean values this is used to chnge the array we created property_stats

CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

In [31]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
     # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range 
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval
    

In [33]:
# median of the data set 
np.median(boston_dataset.target)

# current median in gen as searched from internet 583.3
# converting the price rate of 1970 to current as dollar rates have changed

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

In [34]:
# creating a function which will convert the scalefactor and give the actual price and range

def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
    
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   students_per_classroom=ptratio, 
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [35]:
get_dollar_estimate(rm=6, ptratio=-60, chas=True)

That is unrealistic. Try again.


In [37]:
get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
